<a href="https://colab.research.google.com/github/vvikasreddy/JargonAI/blob/main/Summa_rizzer_on_custom_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader, Dataset

In [4]:
#load the dataset
dataset_path = "/content/Corpus_all.csv"
# try:
#     data = pd.read_csv(dataset_path, encoding="utf-8")
# except UnicodeDe|codeError:
#
data = pd.read_csv(dataset_path, encoding="ISO-8859-1")
# data.drop()

In [5]:
data.iloc[38]

,38
Document,The County Council of West Midlands (M6 Motorw...
Summary,The County Council of West Midlands (M6 Motorw...


In [6]:
print(data.head())

                                            Document  \
0  412.106.1 English is not an official language ...   
1  944.021.1 English is not an official language ...   
2  Title and commencement 1 This order may be cit...   
3  Citation and commencement 1 This order may be ...   
4  Title, commencement and interpretation 1 1 Thi...   

                                             Summary  
0  The Swiss Federal University for Vocational Ed...  
1  The EAER Ordinance on the Declaration for Timb...  
2  The North West Water Authority (Solway Firth) ...  
3  The Trafford Park Development Corporation (Are...  
4  The North West Water Authority (Returns of Eel...  


In [7]:
#extract documents and summaries
data = data.dropna(subset=["Document", "Summary"])#ensure no missing values
data.reset_index(drop=True, inplace=True)

input_texts = data["Document"].tolist()
target_texts = data["Summary"].tolist()

In [8]:
print(input_texts[0])
print(target_texts[0])

412.106.1 English is not an official language of the Swiss Confederation. This translation is provided for information purposes only, has no legal force and may not be relied on in legal proceedings. Ordinance on the Swiss Federal University for Vocational Education and Training(SFUVET Ordinance)of 18 June 2021 (Status as of 1 August 2021)The Swiss Federal Council,on the basis of Article 35 of the SFUVET Act of 25 September 20201,ordains:1 SR 412.106Art. 1 Registered location The Swiss Federal University for Vocational Education and Training (SFUVET) shall be based in Zollikofen.Art. 2 Regional campuses SFUVET shall offer its services through three regional campuses: one in the German-speaking region, one in the French-speaking region and one in the Italian-speaking region of Switzerland.Art. 3 Federal Council's strategic objectives The Federal Department of Economic Affairs, Education and Research (EAER) shall submit SFUVET's strategic objectives drafted by the Federal Council to the 

In [9]:
#initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
#define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenizer, input_texts, target_texts, max_input_length=512, max_target_length=128):
        self.tokenizer = tokenizer
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_encoding = self.tokenizer(
            self.input_texts[idx],
            max_length=self.max_input_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        target_encoding = self.tokenizer(
            self.target_texts[idx],
            max_length=self.max_target_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": target_encoding["input_ids"].squeeze(),
        }



In [26]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_input_texts, test_input_texts, train_target_texts, test_target_texts = train_test_split(
    input_texts, target_texts, test_size=0.1, random_state=42
)

# Create datasets for train, validation, and test
train_dataset = CustomDataset(tokenizer, train_input_texts, train_target_texts)
test_dataset = CustomDataset(tokenizer, test_input_texts, test_target_texts)

batch_size = 32
# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1 shuffle=False)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-26-a5d300f662fc>, line 15)

In [12]:
#set up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [34]:
#training loop
from tqdm import tqdm

loss = float('inf')

epochs = 50
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_dataloader)}")

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(test_dataloader)}")

100%|██████████| 2/2 [00:00<00:00,  2.77it/s]


Epoch 1/50, Loss: 2.9415303468704224
Epoch 1/50, Validation Loss: 2.383004903793335


100%|██████████| 2/2 [00:00<00:00,  2.97it/s]


Epoch 2/50, Loss: 1.9736133217811584
Epoch 2/50, Validation Loss: 2.3756473064422607


100%|██████████| 2/2 [00:00<00:00,  2.98it/s]


...

Epoch 48/50, Loss: 0.7354615032672882
Epoch 48/50, Validation Loss: 2.8049659729003906


100%|██████████| 2/2 [00:00<00:00,  2.98it/s]


Epoch 49/50, Loss: 0.7090731561183929
Epoch 49/50, Validation Loss: 2.838202714920044


100%|██████████| 2/2 [00:00<00:00,  2.99it/s]

Epoch 50/50, Loss: 0.6863137185573578
Epoch 50/50, Validation Loss: 2.8721208572387695


In [49]:
#evaluation
def evaluate_model(dataset, tokenizer, model):
    model.eval()
    #dataloader = DataLoader(dataset, batch_size=16)
    predictions = []
    references = []
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=256)
            predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
            references.extend([tokenizer.decode(ref, skip_special_tokens=True) for ref in batch["labels"]])
    return predictions, references

In [50]:
#run evaluation
predictions, references = evaluate_model(test_dataset, tokenizer, model)

Evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.64s/it]


In [16]:
predictions[2]

"The Federal Assembly of the Swiss Confederation,the Federal Assembly of the Swiss Confederation,considered the Federal Council Dispatch on 18 August 20102,decrees:SR 1012 BBl 2010 5549Art. Subject of recognition The Federal Council is authorised to recognise agreements between private institutions for the avoidance of double taxation with respect to taxes on income and capital, provided the conclusion of an international treaty on the same issue is not an option.Art. Requirements The recognition of an agreement requires that the agreement is compatible with Switzerland's agreement policy for the avoidance of double taxation; if the agreement has been infringed in a serious manner; orc.withdrawal recognition is required to safeguard Switzerland's interests.The agreement is published with the decision on recognition. The referendum is scheduled for the 15 November 2011 and is a referendum."

In [17]:
references[2]

'This Act grants the Federal Council the authority to recognize private agreements between institutions for the avoidance of double taxation on income and capital, when an international treaty on the same issue is not feasible. Recognition of such agreements requires reciprocity, alignment with Switzerlands tax policy, and approval from both the National Council and the Council of States. The Federal Council may withdraw recognition if reciprocity is no longer guaranteed, if the agreement is seriously violated, or if withdrawal is necessary to protect Switzerlands interests. Once recognized, the agreement applies nationwide. The decision and the agreement are published in the Federal Gazette.'

In [18]:
!pip install rouge_score
!pip install bert_score


In [52]:

from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Calculate ROUGE Scores
def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}
    for ref, pred in zip(references, predictions):

        scores = scorer.score(ref, pred)
        for key in rouge_scores:
            rouge_scores[key].append(scores[key].fmeasure)
    return {key: sum(values) / len(values) for key, values in rouge_scores.items()}

rouge_scores = calculate_rouge(predictions, references)
print(f"ROUGE Scores: {rouge_scores}")

# Calculate BERT Scores
def calculate_bert_score(predictions, references):
    P, R, F1 = bert_score(predictions, references, model_type="bert-base-uncased", lang="en")
    return {
        "Precision": P.mean().item(),
        "Recall": R.mean().item(),
        "F1": F1.mean().item()
    }

bert_scores = calculate_bert_score(predictions, references)
print(f"BERT Scores: {bert_scores}")


ROUGE Scores: {'rouge1': 0.35677993516392836, 'rouge2': 0.12389001827567678, 'rougeL': 0.2454581578327401}
BERT Scores: {'Precision': 0.6105272173881531, 'Recall': 0.5724552273750305, 'F1': 0.5900307893753052}


In [51]:
import textwrap

# Set the desired line width
line_width = 145  # Adjust this as needed for your screen

for indx, val in enumerate(predictions):
    print(f"Reference {indx + 1}:\n{textwrap.fill(references[indx], width=line_width)}")
    print(f"Prediction {indx + 1}:\n{textwrap.fill(val, width=line_width)}")
    print("-" * 30)


Reference 1:
The Swiss Federal University for Vocational Education and Training (SFUVET) is based in Zollikofen and has 3 campuses in Switzerland. These
campuses are separated by the language spoken in the region, which are German, French, and Italilan. It colabortates with industry groups and
regional authorities to plan courses, services, and research. The government sets their goals and consults with national organikzations like
trade unions and employers associations. Old rules have been replaced on August 1, 2021, but a few parts will continue to be active until the end
of 2021.
Prediction 1:
The SFUVET Ordinance is established on June 18, 2020. It meets with national and regional advisory boards, planning training courses, and other
relevant research areas. Its strategic objectives include a coordinated approach to strategic planning, training development, training
development, and the establishment of research and development facilities. Its mandated transitional provisions, inc

In [25]:
for i in references:
  print(i)

The Swiss Federal University for Vocational Education and Training (SFUVET) is based in Zollikofen and has 3 campuses in Switzerland. These campuses are separated by the language spoken in the region, which are German, French, and Italilan. It colabortates with industry groups and regional authorities to plan courses, services, and research. The government sets their goals and consults with national organikzations like trade unions and employers associations. Old rules have been replaced on August 1, 2021, but a few parts will continue to be active until the end of 2021.
The Data Protection (Miscellaneous Subject Access Exemptions) Order 1987 exempts certain personal data from the right of access under Part IV of the Data Protection Act 1984. This applies to data where disclosure is restricted or prohibited by specific laws listed in the schedule to the order. These restrictions are intended to protect the interests of individuals or others. The order comes into effect on November 11, 

In [33]:
len(predictions[3])

127